In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import savgol_filter
import random  
import time
import io
import h5py
import tensorflow as tf
from tensorflow.keras.datasets import mnist     # MNIST dataset is onderdeel van Keras
from tensorflow.keras.models import Sequential  # Het type neuraal netwerk dat we gaan gebruiken
from tensorflow.keras.layers import Dense, Dropout, Activation  # Verschillende type lagen die we gaan gebruiken
from tensorflow.keras.optimizers import SGD, Adam, schedules
from tensorflow.keras import utils


configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)

# Data prep

In [2]:
from PIL import Image

def get_pixels_val(image_path): # bron: https://stackoverflow.com/questions/138250/how-to-read-the-rgb-value-of-a-given-pixel-in-python
    """Get a numpy array of an image so that one can access values[x][y]."""
    image = Image.open(image_path, "r")
    pixel_values = np.array(image)

    # width crop
    pixel_values = pixel_values[54:-43]
    
    # height crop
    pv = list()
    for c, h in enumerate(pixel_values):
        pv.append(pixel_values[c, 35:-36])
    
    return np.array(pv)

# Create transformed and halved csv file

In [3]:
# opent de og sat data file
with open(r"D:\FARM_data\Soil_Spectra\spectra_ BE .csv") as f:
    # maakt csv reader aan
    reader = csv.reader(f)
    # Open 
    with open(r"D:\FARM_data\Soil_Spectra_s1\spectra_ BE.csv", 'w', newline='') as file:
        writer = csv.writer(file)
        for c, row in enumerate(reader):
            if c == 0:
                writer.writerow(row[:5] + row[205:-200:2])
            else:
                x = np.array(row[205:-200:2], dtype='float64')
                reflectance = 10**(-x)
                writer.writerow(row[:5] + list(reflectance))


In [7]:
with open(r"D:\FARM_data\Soil_Spectra_s1\spectra_ BE.csv") as fi:
    re = csv.reader(fi)
    lst = list()
    start = time.time()
    row1= np.array(next(re)[5:])
    
    with open(r"D:\FARM_data\Soil_Spectra_s1\spectra_ BE_array.npz", "wb") as fil:
        for c, ro in enumerate(re):
            if c == 100:
                break
            reflectance = np.array(ro[5:])
            r = savgol_filter(reflectance, 11, 2)

            f, t, Sxx = signal.spectrogram(r, 1)

            sxx_sdv = np.array([[ (Sxx[i][j] - np.mean(Sxx[i]) ) / np.std(Sxx[i]) for j in range(Sxx.shape[1])] for i in range(Sxx.shape[0])])
            
            fig, ax = plt.subplots()
            ax.axis('off')
            ax.pcolormesh(t, f, sxx_sdv, shading='gouraud')#, shading='gouraud'
            plt.plot()
            io_buf = io.BytesIO()
            fig.savefig(io_buf, format='rgba', dpi=72)
            io_buf.seek(0)
            img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                                 newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
            io_buf.close()
            plt.close()


#             array = img_arr[35:-36]
#             array = array[:, 54:-43]
#             array = np.delete(array[:], -1, -1)
# #                 print(array)
#             lst.append(array)
#         print(len(lst))
            
# -80 sec (100)
#             plt.savefig(f'image.jpeg')

#             img = Image.open('image.jpeg')
#             print(type(img))
# -3 sec (100)
#             array = np.array(img)
#             array = array[35:-36]
#             array = array[:, 54:-43]
# - 3 sec (100)
#             lst.append(array)
#         np.savez(fil, *lst)
    print(time.time() - start)


# TODO: ervoor zorgen dat sneller csv gevuld word (met batches miss werken), ervoor zorgen dat het goed geprint word
# npz = np.load(r"D:\FARM_data\Soil_Spectra_s1\spectra_ BE_array.npz")

3.800016164779663


In [5]:
f = h5py.File("D:\FARM_data\Soil_Spectra_s1\mytestfile.hdf5", "w")

In [6]:
f.create_dataset('BE', data=lst)

<HDF5 dataset "BE": shape (100, 217, 335, 3), type "|u1">

In [7]:
f.close()

In [8]:
hf = h5py.File('D:\FARM_data\Soil_Spectra_s1\mytestfile.hdf5', 'a')

In [9]:
n1 = hf.get('BE')

In [10]:
print(n1)

<HDF5 dataset "BE": shape (100, 217, 335, 3), type "|u1">


In [14]:
# TODO: Labelen van Spectogram arrays + opsplitsen train en test?(kan miss ook met train_test_split), 
# Het opstellen van een model

In [10]:
# # Bron: https://www.pythoninformer.com/python-libraries/numpy/numpy-and-images/
# img = Image.open('image.jpeg')
# array = np.array(img)
# array = array[35:-36] #54:-43
# array = array[:, 54:-43]
# invimg = Image.fromarray(array)
# invimg.save('TRANSFORM.jpeg')